# Django REST Framework: HyperlinkedModelSerializer Example

This notebook demonstrates how to use `HyperlinkedModelSerializer` to create a RESTful API with hyperlinked relationships.

In [ ]:
# Step 0: Setup Django inside this notebook (if running outside project context)
import os
import django
from pathlib import Path

# Simulate project root
BASE_DIR = Path.cwd() / "drf_hyperlinked_demo"
os.makedirs(BASE_DIR, exist_ok=True)

# Create dummy settings for this notebook
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "drf_hyperlinked_demo.settings")

# Initialize Django (if running this outside a Django project)
try:
    django.setup()
except:
    pass

## Step 1: Define Models (`models.py`)

In [ ]:
# models.py
from django.db import models

class Author(models.Model):
    name = models.CharField(max_length=100)

    def __str__(self):
        return self.name

class Book(models.Model):
    title = models.CharField(max_length=200)
    author = models.ForeignKey(Author, related_name='books', on_delete=models.CASCADE)

    def __str__(self):
        return self.title

## Step 2: Define HyperlinkedModelSerializer Classes (`serializers.py`)

In [ ]:
# serializers.py
from rest_framework import serializers
from .models import Author, Book

class AuthorSerializer(serializers.HyperlinkedModelSerializer):
    books = serializers.HyperlinkedRelatedField(
        many=True,
        read_only=True,
        view_name='book-detail'
    )

    class Meta:
        model = Author
        fields = ['url', 'id', 'name', 'books']

class BookSerializer(serializers.HyperlinkedModelSerializer):
    author = serializers.HyperlinkedRelatedField(
        queryset=Author.objects.all(),
        view_name='author-detail'
    )

    class Meta:
        model = Book
        fields = ['url', 'id', 'title', 'author']

## Step 3: Define ViewSets (`views.py`) and Register Router (`urls.py`)

In [ ]:
# views.py
from rest_framework import viewsets
from .models import Author, Book
from .serializers import AuthorSerializer, BookSerializer

class AuthorViewSet(viewsets.ModelViewSet):
    queryset = Author.objects.all()
    serializer_class = AuthorSerializer

class BookViewSet(viewsets.ModelViewSet):
    queryset = Book.objects.all()
    serializer_class = BookSerializer

In [ ]:
# urls.py
from django.urls import include, path
from rest_framework.routers import DefaultRouter
from .views import AuthorViewSet, BookViewSet

router = DefaultRouter()
router.register(r'authors', AuthorViewSet)
router.register(r'books', BookViewSet)

urlpatterns = [
    path('', include(router.urls)),
]

## Step 4: Example Output (GET `/authors/1/`)

```json
{
  "url": "http://localhost:8000/authors/1/",
  "id": 1,
  "name": "J. R. R. Tolkien",
  "books": [
    "http://localhost:8000/books/1/",
    "http://localhost:8000/books/2/"
  ]
}
```

## 🔍 Summary

| Feature | Description |
|---------|-------------|
| `HyperlinkedModelSerializer` | Uses URLs instead of primary keys or slugs |
| `url` field | Auto-included for hyperlinking each instance |
| `HyperlinkedRelatedField` | Used for related objects as hyperlinks |
| `DefaultRouter` | Automatically maps viewsets to URLs |

This is ideal for RESTful APIs that follow the **HATEOAS** principle — where clients can explore the API using hyperlinks.